### Imports

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from feature_baggingV2 import FeatureBaggingWithHyperparamTuning
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import datetime
from sklearn.preprocessing import MinMaxScaler

import os, sys, gc, time, warnings, pickle, psutil, random

### Función para escalar/desescalar y métrica

In [36]:
# Función para escalar y devolver una serie
def minmax_scale_group(group):
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Almacenar el escalador para este grupo
    return pd.Series(scaled_values, index=group.index)

# Función para desescalar y devolver una serie
def inverse_minmax_scale_group(group):
    scaler = scalers[group.name]
    inversed_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    return pd.Series(inversed_values, index=group.index)

In [37]:
# Definir la métrica personalizada
def multinacional_metric(y_true, y_pred):
    return abs(sum(y_true - y_pred)) / sum(y_true)

### Archivos

In [38]:
DATOS_DIR = 'C:\\Users\\ccarl\\Downloads\\Labo3\\'
df_sell_in = pd.read_csv(DATOS_DIR+'sell-in.txt', sep='\t')
df_predecir = pd.read_csv(DATOS_DIR+'productos_a_predecir.txt', sep='\t')
df_tb_stocks = pd.read_csv(DATOS_DIR+'tb_stocks.txt', sep='\t')
df_tb_productos = pd.read_csv(DATOS_DIR+'tb_productos_descripcion.txt', sep='\t')

Si quiero predecir algun mes en especifico y filtrar los productos con más de 3 meses de venta

In [39]:
#df_sell_in = df_sell_in[df_sell_in['periodo']<201808]
#df_sell_in['tn_2'] = np.where(df_sell_in['periodo'].isin([201805, 201804]), np.nan, df_sell_in['tn_2']) 
#df_sell_in = df_sell_in[df_sell_in['periodo']<201806]
#sales_counts = df_sell_in.groupby('product_id')['periodo'].count() # Contar el número de meses de ventas por product_id
#products_with_more_than_3_months = sales_counts[sales_counts > 3].index # Filtrar los productos con más de 3 meses de ventas
#df_sell_in = df_sell_in[df_sell_in['product_id'].isin(products_with_more_than_3_months)] # Filtrar el DataFrame original para incluir solo estos productos

### Preprocesamiento

In [40]:
df_tb_stocks['periodo'] = pd.to_datetime(df_tb_stocks['periodo'], format='%Y%m')
df_tb_stocks['periodo'] = df_tb_stocks['periodo'] - pd.DateOffset(months=1) #mes diferente en stock


df_tb_stocks['product_id'] = df_tb_stocks['product_id'].astype(int)
df_tb_stocks['stock_final'] = df_tb_stocks['stock_final'].astype(float)
df_tb_productos['product_id'] = df_tb_productos['product_id'].astype(int)
df_tb_productos['sku_size'] = df_tb_productos['sku_size'].astype(int)
df_sell_in['periodo'] = pd.to_datetime(df_sell_in['periodo'], format='%Y%m')
df_sell_in['product_id'] = df_sell_in['product_id'].astype(int)
df_sell_in['customer_id'] = df_sell_in['customer_id'].astype(int)
df_sell_in['cust_request_qty'] = df_sell_in['cust_request_qty'].astype(int)
df_sell_in['cust_request_tn'] = df_sell_in['cust_request_tn'].astype(float)
df_sell_in['tn'] = df_sell_in['tn'].astype(float)
df_sell_in['plan_precios_cuidados'] = df_sell_in['plan_precios_cuidados'].astype(bool)

### Consolidar Datos

In [41]:
df_sell_in['tn_2'] = df_sell_in['tn'].shift(-2) #OBTENGO TN 2 MESES ADELANTE (CLASE FINAL)

In [42]:
# Join tb_productos to sell_in on product_id
df_sell_in_merged = pd.merge(df_sell_in, df_tb_productos, on='product_id', how='left')
# Join tb_stocks to sell_in_merged on both product_id and periodo
df_final = pd.merge(df_sell_in_merged, df_tb_stocks, on=['product_id', 'periodo'], how='left')

#Convertir 'periodo' a formato de fecha y Calcular el trimestre desde 'periodo'
df_final['fecha'] = pd.to_datetime(df_final['periodo'], format='%Y%m')
df_final["trimestre"] = df_final.periodo.dt.quarter

#Establecer 'fecha' como índice y convertir a período mensual
df_final.set_index('fecha', inplace=True)
df_final.index = df_final.index.to_period('M')

In [43]:
df_final

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,tn_2,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final,trimestre
fecha,,,,,,,,,,,,,,,,
2017-01,2017-01-01,10234,20524,False,2,0.05300,0.05300,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1
2017-01,2017-01-01,10032,20524,False,1,0.13628,0.13628,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1
2017-01,2017-01-01,10217,20524,False,1,0.03028,0.03028,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1
2017-01,2017-01-01,10125,20524,False,1,0.02271,0.02271,0.01514,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1
2017-01,2017-01-01,10012,20524,False,11,1.54452,1.54452,0.10600,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12,2019-12-01,10105,20853,False,1,0.02230,0.02230,0.02898,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1,NaN,4
2019-12,2019-12-01,10092,20853,False,1,0.00669,0.00669,0.01561,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1,NaN,4
2019-12,2019-12-01,10006,20853,False,7,0.02898,0.02898,0.01561,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1,NaN,4


### Clase de LightGBM

In [44]:
df_final['diff_tn_tn2'] =  df_final['tn_2'] - df_final['tn'] #NUEVA CLASE

In [45]:
df_final

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,tn_2,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final,trimestre,diff_tn_tn2
fecha,,,,,,,,,,,,,,,,,
2017-01,2017-01-01,10234,20524,False,2,0.05300,0.05300,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1,-0.02272
2017-01,2017-01-01,10032,20524,False,1,0.13628,0.13628,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1,-0.11357
2017-01,2017-01-01,10217,20524,False,1,0.03028,0.03028,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1,1.51424
2017-01,2017-01-01,10125,20524,False,1,0.02271,0.02271,0.01514,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1,-0.00757
2017-01,2017-01-01,10012,20524,False,11,1.54452,1.54452,0.10600,HC,VAJILLA,Cristalino,Importado,500.0,Abrillantador,NaN,1,-1.43852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12,2019-12-01,10105,20853,False,1,0.02230,0.02230,0.02898,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1,NaN,4,0.00668
2019-12,2019-12-01,10092,20853,False,1,0.00669,0.00669,0.01561,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1,NaN,4,0.00892
2019-12,2019-12-01,10006,20853,False,7,0.02898,0.02898,0.01561,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,Sabor 1,NaN,4,-0.01337


## FE

### Lags

In [46]:
# Create lag variables for 'cust_request_qty', 'cust_request_tn', and 'tn'
n_lags = 36
for lag in range(1, n_lags + 1):
    df_final[f'cust_request_qty_lag_{lag}'] = df_final['cust_request_qty'].shift(lag)
    df_final[f'cust_request_tn_lag_{lag}'] = df_final['cust_request_tn'].shift(lag)
    df_final[f'stock_final_lag_{lag}'] = df_final['stock_final'].shift(lag)
    df_final[f'tn_lag_{lag}'] = df_final['tn'].shift(lag)

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\648479522.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'cust_request_tn_lag_{lag}'] = df_final['cust_request_tn'].shift(lag)
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\648479522.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'stock_final_lag_{lag}'] = df_final['stock_final'].shift(lag)
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\648479522.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

In [47]:
#DELTA DE LAGS 2 ANTERIORES

for lag in range(1, n_lags-1):
    df_final[f'delta_cust_request_tn_{lag}_{lag+2}'] = (
        df_final[f'cust_request_tn_lag_{lag+2}'] - df_final[f'cust_request_tn_lag_{lag}']
    )
    df_final[f'delta_stock_final_{lag}_{lag+2}'] = (
        df_final[f'stock_final_lag_{lag+2}'] - df_final[f'stock_final_lag_{lag}']
    )
    df_final[f'delta_tn_{lag}_{lag+2}'] = (
        df_final[f'tn_lag_{lag+2}'] - df_final[f'tn_lag_{lag}']
    )

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\260571944.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_cust_request_tn_{lag}_{lag+2}'] = (
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\260571944.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_stock_final_{lag}_{lag+2}'] = (
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\260571944.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

In [48]:
#DELTA DE LAGS 1 ANTERIOR

for lag in range(1, n_lags):
    df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
        df_final[f'cust_request_tn_lag_{lag+1}'] - df_final[f'cust_request_tn_lag_{lag}']
    )
    df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
        df_final[f'stock_final_lag_{lag+1}'] - df_final[f'stock_final_lag_{lag}']
    )
    df_final[f'delta_tn_{lag}_{lag+1}'] = (
        df_final[f'tn_lag_{lag+1}'] - df_final[f'tn_lag_{lag}']
    )

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\1785296116.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\1785296116.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\1785296116.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

In [49]:
#Mes actual / (lag 2 + lag 3) 

# Calcular el ratio del valor actual con respecto a la suma de los dos lags anteriores
for lag in range(3, n_lags + 1):
    df_final[f'ratio_cust_request_tn_{lag}'] = (
        df_final[f'cust_request_tn_lag_{lag}'] / (
            df_final[f'cust_request_tn_lag_{lag-1}'] + df_final[f'cust_request_tn_lag_{lag-2}']
        )
    )
    df_final[f'ratio_stock_final_{lag}'] = (
        df_final[f'stock_final_lag_{lag}'] / (
            df_final[f'stock_final_lag_{lag-1}'] + df_final[f'stock_final_lag_{lag-2}']
        )
    )
    df_final[f'ratio_tn_{lag}'] = (
        df_final[f'tn_lag_{lag}'] / (
            df_final[f'tn_lag_{lag-1}'] + df_final[f'tn_lag_{lag-2}']
        )
    )

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\2974401302.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_cust_request_tn_{lag}'] = (
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\2974401302.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_stock_final_{lag}'] = (
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\2974401302.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consid

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\2974401302.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_tn_{lag}'] = (
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\2974401302.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_cust_request_tn_{lag}'] = (
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\2974401302.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

In [50]:
df_final.reset_index(inplace=True)

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3586226101.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final.reset_index(inplace=True)


### Medias Móviles

In [51]:
#MEDIAS MOVILES
rolling_windows = [3, 6, 9, 12, 24, 36]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for window in rolling_windows:
    df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3749147370.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3749147370.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\ccarl\AppData\Local\Tem

### FE

In [52]:
# Datetime features
df_final['year'] = df_final['periodo'].dt.year
df_final['month'] = df_final['periodo'].dt.month
df_final['quarter'] = df_final.periodo.dt.quarter

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3544365912.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['year'] = df_final['periodo'].dt.year
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3544365912.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['month'] = df_final['periodo'].dt.month
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3544365912.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

In [53]:
#Variables Dummies si es el max o el min de cierta cantidad de meses
months = [3, 6, 9, 12]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for i in months:
    df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
    df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())
    # Crear las dummies
    df_final[f'dummy_max_{i}m'] = np.where(df_final['tn'] == df_final[f'max_{i}m'], 1, 0)
    df_final[f'dummy_min_{i}m'] = np.where(df_final['tn'] == df_final[f'min_{i}m'], 1, 0)

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3137070357.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3137070357.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3137070357.py:9: Performan

In [54]:
#Calcular ventas por trimestre
df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')
#Calcular ventas por trimestre por cliente
df_final['tn_trimestre_customer'] = df_final.groupby(['trimestre','customer_id', 'product_id'])['tn'].transform('sum')

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3111183930.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')
C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\3111183930.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_trimestre_customer'] = df_final.groupby(['trimestre','customer_id', 'product_id'])['tn'].transform('sum')


In [55]:
df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')

C:\Users\ccarl\AppData\Local\Temp\ipykernel_11516\4104376824.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')


### FE variables externas

In [62]:
# Leer el archivo exportado
df_exported = pd.read_excel('23variables_externas.xlsx')

# Asegúrate de que las columnas de fecha estén en el formato datetime
df_exported['fecha'] = pd.to_datetime(df_exported['fecha'])

# Unir los DataFrames por la columna de fecha
df_merged = pd.merge(df_final, df_exported, on='fecha', how='left')

# Mostrar el DataFrame unido
print(df_merged.head())

In [56]:
df_final

,fecha,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,tn_2,cat1,...,min_9m,dummy_max_9m,dummy_min_9m,max_12m,min_12m,dummy_max_12m,dummy_min_12m,tn_trimestre,tn_trimestre_customer,tn_product_id
0,2017-01,2017-01-01,10234,20524,False,2,0.05300,0.05300,0.03028,HC,...,0.05300,1,1,0.05300,0.05300,1,1,64.00631,0.12871,6.48085
1,2017-01,2017-01-01,10032,20524,False,1,0.13628,0.13628,0.02271,HC,...,0.05300,1,0,0.13628,0.05300,1,0,64.00631,1.36281,6.48085
2,2017-01,2017-01-01,10217,20524,False,1,0.03028,0.03028,1.54452,HC,...,0.03028,0,1,0.13628,0.03028,0,1,64.00631,0.09842,6.48085
3,2017-01,2017-01-01,10125,20524,False,1,0.02271,0.02271,0.01514,HC,...,0.02271,0,1,0.13628,0.02271,0,1,64.00631,0.03785,6.48085
4,2017-01,2017-01-01,10012,20524,False,11,1.54452,1.54452,0.10600,HC,...,0.02271,1,0,1.54452,0.02271,1,0,64.00631,7.38191,6.48085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945813,2019-12,2019-12-01,10105,20853,False,1,0.02230,0.02230,0.02898,PC,...,0.00446,0,0,0.62426,0.00446,0,0,18.55436,0.10479,2.89842
2945814,2019-12,2019-12-01,10092,20853,False,1,0.00669,0.00669,0.01561,PC,...,0.00446,0,0,0.62426,0.00446,0,0,18.55436,0.01561,2.89842
2945815,2019-12,2019-12-01,10006,20853,False,7,0.02898,0.02898,0.01561,PC,...,0.00446,0,0,0.62426,0.00446,0,0,18.55436,0.66662,2.89842
2945816,2019-12,2019-12-01,10018,20853,False,4,0.01561,0.01561,NaN,PC,...,0.00446,0,0,0.62426,0.00446,0,0,18.55436,0.31213,2.89842


In [57]:
df_final['plan_precios_cuidados'] =df_final['plan_precios_cuidados'].astype(bool)
#df_final['dias_fin_trimestre'] = df_final['dias_fin_trimestre'].dt.days.astype(int) #TARDA!
df_final = df_final.drop(columns=['fecha'])

SyntaxError: invalid syntax (2930202222.py, line 1)

## Inicio Train

In [ ]:
# #Cambiar las variables categoricas y hacer one-hot encoding

# df_final["cat1"] = df_final["cat1"].astype("category")
# df_final["cat2"] = df_final["cat2"].astype("category")
# df_final["cat3"] = df_final["cat3"].astype("category")
# df_final["brand"] = df_final["brand"].astype("category")
# df_final["descripcion"] = df_final["descripcion"].astype("category")

# # Encode categorical variables explicitly. One-hot encoding
# cat1_dummies = pd.get_dummies(df_final['cat1'], prefix='cat1', drop_first=True)
# cat2_dummies = pd.get_dummies(df_final['cat2'], prefix='cat2', drop_first=True)
# cat3_dummies = pd.get_dummies(df_final['cat3'], prefix='cat3', drop_first=True)
# brand_dummies = pd.get_dummies(df_final['brand'], prefix='brand', drop_first=True)
# descripcion_dummies = pd.get_dummies(df_final['descripcion'], prefix='descripcion', drop_first=True)

# # Concatenate the dummy variables to the DataFrame and drop the original categorical columns
# df_final= pd.concat([df_final, cat1_dummies, cat2_dummies, cat3_dummies, brand_dummies, descripcion_dummies], axis=1)
# df_final.drop(columns=['cat1', 'cat2', 'cat3', 'brand'], inplace=True)

# df_final.set_index('periodo', inplace=True)
# df_final.index = df_final.index.to_period('M')
# df_final.sort_index(inplace=True)


In [ ]:
scalers = {}

df_final['tn'] = df_final.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
df_final['tn_2'] = df_final.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado

In [ ]:
data = df_final 

In [ ]:
import lightgbm as lgb
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
        'metric':'rmse',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.2,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5
}

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [ ]:
VERSION = '1'                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED  

#LIMITS and const
TARGET      = 'tn_2'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = datetime.datetime(year=2019,month=12,day=1) - pd.DateOffset(months=6)           # End day of our train set
PRED = '2019-12-01'
P_HORIZON   = pd.DateOffset(months=1)                  # Prediction horizon
USE_AUX     = False               # Use or not pretrained models

In [ ]:
PRODUCTS_IDS = df_predecir['product_id'].unique()
DESCRIPCION_TYPES = df_tb_productos['descripcion'].unique()

In [ ]:
all_preds = pd.DataFrame()

In [ ]:
for descripcion_type in DESCRIPCION_TYPES:
    
    grid_df = data[data['descripcion']==descripcion_type]
    train_mask = grid_df['periodo']<=END_TRAIN
    valid_mask = train_mask&(grid_df['periodo']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['periodo']==(PRED)

    train_data_filtered = grid_df[train_mask].drop(columns=['periodo'])
    valid_data_filtered = grid_df[valid_mask].drop(columns=['periodo'])
    preds_data_filtered = grid_df[preds_mask].drop(columns=['periodo'])
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)

    if train_data_filtered.empty:
        # Si el conjunto de entrenamiento está vacío, llenar y_pred con ceros
        preds_data_filtered['prediction'] = 0

    else:
        train_data = lgb.Dataset(train_data_filtered.drop(columns=[TARGET]), 
                                 label=train_data_filtered[TARGET])
        train_data.save_binary('train_data.bin')
        train_data = lgb.Dataset('train_data.bin')
        
        valid_data = lgb.Dataset(valid_data_filtered.drop(columns=[TARGET]), 
                           label=valid_data_filtered[TARGET])
        
        if valid_data_filtered.empty:
            estimator = lgb.train(lgb_params,
                            train_data,
                            num_boost_round=3600)

        else:
            estimator = lgb.train(lgb_params,
                            train_data,
                            num_boost_round=3600,
                            valid_sets=[train_data, valid_data],
                            callbacks=[lgb.early_stopping(stopping_rounds=50)])
        
    
        # Realizar predicciones
        y_pred = estimator.predict(preds_data_filtered.drop(columns=[TARGET]), num_iteration=estimator.best_iteration)
        
        # Agregar las predicciones al DataFrame
        preds_data_filtered['prediction'] = y_pred
        #preds_data_filtered['product_id'] = product_id

        # Save model - it's not real '.bin' but a pickle file
        # estimator = lgb.Booster(model_file='model.txt')
        # can only predict with the best iteration (or the saving iteration)
        # pickle.dump gives us more flexibility
        # like estimator.predict(TEST, num_iteration=100)
        # num_iteration - number of iteration want to predict with, 
        # NULL or <= 0 means use best iteration
            # "Keep" models features for predictions
        model_name = 'lgb_model_'+descripcion_type +'_v'+(VERSION)+'.bin'
        pickle.dump(estimator, open(model_name, 'wb'))
    
        # Remove temporary files and objects 
        # to free some hdd space and ram memory
        !rm train_data.bin
        del train_data, valid_data, estimator
        gc.collect()

    preds_data_filtered['tn_2'] = preds_data_filtered.groupby('product_id')['tn_2'].transform(inverse_minmax_scale_group)
    preds_data_filtered = preds_data_filtered.groupby(['product_id'])['tn_2'].agg('sum')
    preds_data_filtered = preds_data_filtered.reset_index()
    preds_data_filtered = preds_data_filtered.rename(columns={'tn_2': 'tn'}) #PREDICCION TN DE 2 MESES ADELANTE

    all_preds = pd.concat([all_preds, preds_data_filtered], axis=0)


# Guardar todas las predicciones en un archivo CSV
all_preds.to_csv('all_predictions.csv', index=False)

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: cat1: object, cat2: object, cat3: object, brand: object, descripcion: object

In [ ]:
#data_predecir2
# #SI QUIERO COMPARAR CONTRA julio
# df_sell_in_julio = pd.read_csv('data/sell-in.txt', sep='\t')
# df_sell_in_julio = df_sell_in_julio[df_sell_in_julio['periodo'] == 201807]

# # Contar el número de meses de ventas por product_id
# sales_counts_julio = df_sell_in_julio.groupby('product_id')['periodo'].count()
# products_with_more_than_3_months_julio = sales_counts[sales_counts > 3].index
# df_sell_in_julio = df_sell_in_julio[df_sell_in_julio['product_id'].isin(products_with_more_than_3_months_julio)]

# valores_reales_julio = df_sell_in_julio
# valores_reales_julio = valores_reales_julio.groupby(['product_id'])['tn'].agg('sum')

# # Combinar predicciones y valores reales
# resultados = data_predecir2.merge(valores_reales_julio, on='product_id')

# productos = df_tb_productos.loc[:, ['product_id','cat1']]

# resultados = resultados.merge(productos, on='product_id')

In [ ]:

# Calcular la métrica
#resultados = resultados[resultados['cat1']=='FOODS']
#y_true = resultados['tn']
#y_pred = resultados['pred']
#metric_result = multinacional_metric(y_true, y_pred)
#metric_result
#df_predictions
#resultados